# Final Project -- ddl2: Feature Engineering and Models

### Group: Toxic  
### Member: 
**Qingyi (Lexie) Sun**,  Model and EDA  
**Todd Zhang**,  EDA and Visualization

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from time import time
import datetime
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,KFold
import gc
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import random

## Prepare Data

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data.head()

,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,y_mean_MAP,y_mean_HR
0,1891,1891-3,1,75,22,1,2,0,0,1,80.0,95.0,99.9,49.4,62.5,59.04,80.0
1,1891,1891-3,1,75,22,1,2,0,0,1,80.0,94.9,98.4,48.7,61.3,59.04,80.0
2,1891,1891-3,1,75,22,1,2,0,0,1,80.0,95.0,95.3,48.5,60.3,59.04,80.0
3,1891,1891-3,1,75,22,1,2,0,0,1,80.0,95.0,97.4,48.9,61.3,59.04,80.0
4,1891,1891-3,1,75,22,1,2,0,0,1,76.7,95.7,99.6,50.2,62.8,59.04,80.0


In [4]:
test = pd.read_csv('data_for_test.csv')

In [5]:
test.head(5)

,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5
0,1005,1005-1,0,80,41,4,2,1,0,0,59.0,99.0,118.0,38.0,66.0
1,1005,1005-1,0,80,41,4,2,1,0,0,60.0,98.0,124.0,41.0,71.0
2,1005,1005-1,0,80,41,4,2,1,0,0,60.0,97.0,125.0,42.0,70.0
3,1005,1005-1,0,80,41,4,2,1,0,0,58.0,99.0,123.0,41.0,69.0
4,1005,1005-1,0,80,41,4,2,1,0,0,60.0,100.0,133.0,47.0,78.0


In [6]:
group_mean_data = data.groupby('key').mean().reset_index()
# group_mean_data['stamp'] = pd.to_numeric(group_mean_data['key'].str.split('-').str[1])

In [7]:
group_mean_data

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,y_mean_MAP,y_mean_HR
0,0-28,0,0,72,39,5,1,0,0,0,80.166667,99.733333,113.433333,69.566667,87.000000,86.426667,79.130000
1,1-10,1,1,64,55,9,5,1,0,1,72.100000,99.800000,113.900000,59.600000,76.466667,73.033333,72.920000
2,1-11,1,1,64,55,9,5,1,0,1,71.500000,99.966667,114.066667,59.500000,75.400000,77.560000,72.493333
3,1-14,1,1,64,55,9,5,1,0,1,99.133333,100.000000,133.433333,69.833333,84.966667,82.523333,118.613333
4,1-15,1,1,64,55,9,5,1,0,1,150.200000,100.000000,123.733333,77.100000,89.233333,98.706667,189.720000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44944,999-21,999,0,41,0,1,5,0,0,0,98.200000,93.800000,116.666667,57.166667,73.000000,72.100000,96.116667
44945,999-3,999,0,41,0,1,5,0,0,0,103.233333,98.066667,119.100000,56.233333,73.566667,66.503333,106.463333
44946,999-5,999,0,41,0,1,5,0,0,0,106.300000,94.466667,110.700000,56.433333,71.466667,67.610000,106.866667
44947,999-7,999,0,41,0,1,5,0,0,0,108.200000,98.033333,104.266667,58.166667,71.100000,70.396667,107.026667


In [27]:
group_tail_data = data.groupby('key').tail(1)
# group_tail_data['stamp'] = pd.to_numeric(group_tail_data['key'].str.split('-').str[1])
# group_tail_data = group_tail_data.sort_values(['patient_id', 'stamp'])
# group_tail_data = group_tail_data.groupby(['patient_id']).tail(10)

In [9]:
# group_mean_data = group_mean_data.sort_values(['patient_id', 'stamp'])
# data_mean = group_mean_data.groupby(['patient_id']).tail(10)

In [10]:
# group_tail_data.head(20)
# data_mean.head(20)
# group_mean_data.head(20)

In [28]:
def get_data(data):
    X = data[['gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5']]
    y_lgmap = np.log(data['y_mean_MAP'])
    y_hr = data['y_mean_HR']
    return X, y_lgmap, y_hr

In [29]:
X, y_lgmap, y_hr = get_data(group_tail_data)

#### 1. train-test split

In [30]:
y = pd.concat([y_lgmap, y_hr], axis=1)

In [31]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
ym_train = y_train['y_mean_MAP']
ym_val = y_val['y_mean_MAP']
yh_train = y_train['y_mean_HR']
yh_val = y_val['y_mean_HR']

In [32]:
X_train

,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5
29,1,75,22,1,2,0,0,1,80.000,96.800,79.900,45.300,54.000
59,1,75,22,1,2,0,0,1,80.000,100.000,151.800,63.600,89.700
89,1,75,22,1,2,0,0,1,80.000,99.400,94.700,45.600,57.100
119,1,75,22,1,2,0,0,1,80.000,100.000,157.600,61.000,87.200
149,1,75,22,1,2,0,0,1,80.000,96.000,126.400,50.600,69.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078649,0,82,47,8,6,0,0,0,68.467,96.034,132.335,55.616,79.634
1078679,0,82,47,8,6,0,0,0,67.484,95.417,111.786,51.116,71.150
1078709,0,82,47,8,6,0,0,0,60.766,94.200,147.583,59.349,84.948
1078739,0,82,47,8,6,0,0,0,69.166,94.767,132.084,57.066,79.964


## Models

In [16]:
def fit_evaluate_model(model, X_train, y_train, X_valid, y_valid):
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_valid)
    return model, r2_score(y_valid, y_predicted)
#     return model, r2_score(np.exp(y_valid), np.exp(y_predicted))
#     return np.sqrt(((y_predicted - y_valid) ** 2).mean())

### Linear Regression model for y_mean_HR  
* Using groupby mean data(all key_id).

In [18]:
linear_regression = LinearRegression()

In [19]:
lr, lr_yh_val = fit_evaluate_model(linear_regression, X_train, yh_train, X_val, yh_val)
print("R^2 on validation set of the linear regression model is:", lr_yh_val)

R^2 on validation set of the linear regression model is: 0.9220414878207371


### Random Forest model for y_mean_MAP  
* Using groupby tail data(resample, tail key_id).  
R^2 on validation set of the Random Forest regression model is: 0.8912117057728295
* Using groupby tail data(unsample, all key_id).   
R^2 on validation set of the Random Forest regression model is: 0.9215796526108799

In [33]:
rf_tail = RandomForestRegressor(n_estimators=500,
                           min_samples_leaf=80,
                           random_state=42, 
                           max_depth=10, 
                           criterion='mse', 
                           bootstrap=True,
                           n_jobs=-1)

In [34]:
rf_tail, rf_ym_val = fit_evaluate_model(rf_tail, X_train, ym_train, X_val, ym_val)
print("R^2 on validation set of the Random Forest regression model is:", rf_ym_val)

R^2 on validation set of the Random Forest regression model is: 0.9215796526108799


#### RandomSearch CV to find the best model setting.

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [39]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 19)]
max_depth = [int(x) for x in np.linspace(1, 20, num = 5)]
min_samples_leaf = [10, 20, 30, 50, 80, 100, 150]
bootstrap = True
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000], 'max_depth': [1, 5, 10, 15, 20], 'min_samples_leaf': [10, 20, 30, 50, 80, 100, 150]}


In [41]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X, y_lgmap)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 24.6min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...o

In [43]:
rf_random.best_params_

{'n_estimators': 1000, 'min_samples_leaf': 50, 'max_depth': 10}

In [44]:
rf_best = RandomForestRegressor(n_estimators=1000,
                           min_samples_leaf=50,
                           random_state=42, 
                           max_depth=10, 
                           criterion='mse', 
                           bootstrap=True,
                           n_jobs=-1)

In [45]:
rf_best, rf_ym_val = fit_evaluate_model(rf_best, X_train, ym_train, X_val, ym_val)
print("R^2 on validation set of the Random Forest regression model is:", rf_ym_val)

R^2 on validation set of the Random Forest regression model is: 0.9216267410180505


### NN

In [174]:
X_train

,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5
29,1,75,22,1,2,0,0,1,80.000,96.800,79.900,45.300,54.000
59,1,75,22,1,2,0,0,1,80.000,100.000,151.800,63.600,89.700
89,1,75,22,1,2,0,0,1,80.000,99.400,94.700,45.600,57.100
119,1,75,22,1,2,0,0,1,80.000,100.000,157.600,61.000,87.200
149,1,75,22,1,2,0,0,1,80.000,96.000,126.400,50.600,69.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078649,0,82,47,8,6,0,0,0,68.467,96.034,132.335,55.616,79.634
1078679,0,82,47,8,6,0,0,0,67.484,95.417,111.786,51.116,71.150
1078709,0,82,47,8,6,0,0,0,60.766,94.200,147.583,59.349,84.948
1078739,0,82,47,8,6,0,0,0,69.166,94.767,132.084,57.066,79.964


In [173]:
from sklearn.neural_network import MLPRegressor

In [253]:
hidden_layer_sizes = (13, 6, 1)
activation = ['tanh', 'relu']
solver = ['adam']
alpha = [0.0001, 0.00005, 0.00001]
learning_rate = ['constant', 'adaptive']
# Create the random grid
random_grid = {'hidden_layer_sizes': hidden_layer_sizes,
               'activation': activation,
               'alpha': alpha,
               'solver': solver, 
               'learning_rate':learning_rate,
               'max_iter': [500]}
print(random_grid)

{'hidden_layer_sizes': (13, 6, 1), 'activation': ['tanh', 'relu'], 'alpha': [0.0001, 5e-05, 1e-05], 'solver': ['adam'], 'learning_rate': ['constant', 'adaptive'], 'max_iter': [500]}


In [254]:
mlp = MLPRegressor()

In [255]:
mlp_random = RandomizedSearchCV(estimator = mlp, param_distributions = random_grid, n_iter = 500, cv = 3, verbose=1, random_state=42, n_jobs = -1)
mlp_random.fit(X, y_lgmap)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 36 is smaller than n_iter=500. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   52.0s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                          batch_size='auto', beta_1=0.9,
                                          beta_2=0.999, early_stopping=False,
                                          epsilon=1e-08,
                                          hidden_layer_sizes=(100,),
                                          learning_rate='constant',
                                          learning_rate_init=0.001,
                                          max_iter=200, momentum=0.9,
                                          n_iter_no_change=10,
                                          nesterovs_momentum=True, power_t=0.5,
                                          rando...
                                          validation_fraction=0.1,
                                          verbose=False, warm_start=False),
                   iid='warn', n_iter=500, n_jo

In [256]:
mlp_random.best_params_

{'solver': 'adam',
 'max_iter': 500,
 'learning_rate': 'adaptive',
 'hidden_layer_sizes': 13,
 'alpha': 1e-05,
 'activation': 'tanh'}

In [260]:
mlp = MLPRegressor(hidden_layer_sizes=(13, 6, 1), activation='tanh', solver='adam', max_iter=500, alpha=0.0001)
mlp.fit(X_train, ym_train)

# predict_train = mlp.predict(X_train)
# predict_test = mlp.predict(X_test)

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(13, 6, 1), learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [261]:
ym_val_predict = mlp.predict(X_val)

In [262]:
r2_score(np.exp(ym_val), np.exp(ym_val_predict))

0.8941372488715875

#### --------------------- --------------------- --------------------- --------------------- --------------------- ---------------------

## Predict

In [76]:
sample_submission = pd.read_csv('sample_submission.csv')

In [154]:
### for test hr
group_mean_test_hr = test.groupby('key').mean().reset_index()
Xtest_hr = group_mean_test_hr[['gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5']]

In [139]:
group_tail_test_map = test.groupby('key').tail(1).reset_index()
Xtest_map = group_tail_test_map[['gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5']]

In [63]:
Xtest = test[['gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5']]

In [56]:
# Xtest_hr
# group_mean_test_hr
# Xtest_map
# group_tail_test_map

In [155]:
HR = lr.predict(Xtest_hr)

In [270]:
MAP = mlp.predict(Xtest)
MAP = np.exp(MAP)

In [271]:
MAP_df = pd.DataFrame(MAP)
HR_df = pd.DataFrame(HR)
# sub_19 = pd.concat([test['key'], MAP_df, HR_df], axis=1)
# sub_19.columns = ["key", "y_mean_MAP", 'y_mean_HR']

In [277]:
# make sure key and target matches.
sub_20hr = pd.concat([group_mean_test_hr['key'], HR_df], axis=1)
sub_20map = pd.concat([test['key'], MAP_df], axis=1).groupby('key').tail(1)

In [280]:
sub_20final = sub_20map.join(sub_20hr.set_index('key'), lsuffix='l', on='key')

In [281]:
sub_20final.columns = ["key", "y_mean_MAP", 'y_mean_HR']
sub_20final.to_csv('submission_20.csv', index=0)